# import

In [41]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import gender_guesser.detector as gender
import matplotlib.pyplot as plt 
from datetime import datetime
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [43]:
def predict_sex(name):
    name=str(name).split(' ')[0]
    sex_prediction = gender.Detector(case_sensitive=False)
    sex=sex_prediction.get_gender(name)
    sex_dic={'female':2,"male":1,'unknown':0,'mostly_female':-2 ,'mostly_male':-1}
    return sex


def creation_year(year):
    try:
        dt = datetime.strptime(str(year), '%Y-%m-%d %H:%M:%S')
        return dt.year
    except:
        return 0

def set_sex_code(x):
    x.loc[:,'sex_code']=predict_sex(x['name'])


# load data

In [46]:
data1 = pd.read_csv('cleandata.csv')

y = data1.account_type.values.tolist()
def extract_features(x):
    columns=['statuses','description','location','url','sex_code','verified','created','most_recent_post','following','followers','likes','lists','tweet_language','tweets_this_week','retweeted_count','tweets_by_day_of_week','tweets_by_hour_of_day','account_type']
    x.loc[:,'sex_code']=x.apply(lambda row :predict_sex(row["display_name"]), axis=1)
    print('sex code done')
    x.loc[:,'created'] = x.apply(lambda row :creation_year(row["date_joined"]), axis=1)
    print('created done')
    x.loc[:,'most_recent_post'] = x.apply(lambda row :creation_year(row["most_recent_post"]), axis=1)
    print('most_recent_post done')
    x=x.loc[:,columns]
    return x


In [50]:
data=extract_features(data1)

x=data.loc[:,['statuses','description','location','url','sex_code','verified','created','most_recent_post','following','followers','likes','lists','tweet_language','tweets_this_week','retweeted_count','tweets_by_day_of_week','tweets_by_hour_of_day']]


KeyboardInterrupt: 

split data

In [79]:

x=data.loc[:,['statuses','description','location','url','sex_code','verified','created','most_recent_post','following','followers','likes','lists','tweet_language','tweets_this_week','retweeted_count']]


data.to_csv('features.csv', index=False)

x.to_csv('x.csv', index=False)


In [86]:

clf=RandomForestClassifier()

x=data.loc[:,['statuses','description','location','url','sex_code','verified','created','most_recent_post','following','followers','likes','lists','tweet_language','tweets_this_week','retweeted_count']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42,stratify=y)
clf.fit(x_train,y_train)
y_pred_train = clf.predict(x_train)
y_pred_test = clf.predict(x_test)
print("\n Training Accuracy score:",metrics.accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",metrics.accuracy_score(y_test, y_pred_test))
print("RandomForestClassifier score:",clf.score(x_train, y_train))
print("classification_report \n "+classification_report(y_test, y_pred_test))
cm = metrics.confusion_matrix(y_test, y_pred_test)
# print('Confusion matrix\n', cm)

cm_matrixNB = pd.DataFrame(data=cm, columns=[ 'human', 'bot'], 
                        index=[ 'human', 'bot'])
sns.heatmap(cm_matrixNB, annot=True, fmt='d', cmap='YlGnBu')
plt.show()
   


ValueError: could not convert string to float: 'passionate decent human'